# Simulating a SNN

@[Xiaoyu Chen](mailto:c-xy17@tsinghua.org.cn)

Spiking neural networks (SNN) are one of the most important tools to study brian dynamcis in computational neuroscience. They simulate the biological processes of information transmission in the brain, including the change of membrane potentials, neuronal firing, and synaptic transmission. In this section, we will illustrate how to build and simulate a SNN.

Before we start, the BrainPy package should be imported:

In [4]:
import brainpy as bp

bp.math.set_platform('cpu')

## Building a SNN

Let's try to build a E-I balance network. The structure of a E-I balance network is as follows:

<img src="../_static/E_I_balance_network.png" width="400 px">
<center>Illustration of an E-I Balance Network<center>

A E-I balance network is composed of two neuron groups and the synaptic connections between them. Specifically, they include:
1. a group of excitatory neurons (E),
2. a group of inhibitory neurons (I),
3. synaptic connections within the excitatory and inhibitory neuron groups, respectively, and 
4. the inter-connections between these two groups.

Let's choose the simplest yet the most canonical neuron model, the Leaky Integrate-and-Fire (LIF) model, to build the excitatory and inhibitory neuron groups:

In [5]:
E = bp.dyn.LIF(3200, V_rest=-60., V_th=-50., V_reset=-60., tau=20., tau_ref=5., method='exp_auto')
I = bp.dyn.LIF(800, V_rest=-60., V_th=-50., V_reset=-60., tau=20., tau_ref=5., method='exp_auto')

When defining the LIF neuron group, the parameters can be set according to users' need. The first parameter denotes the number of neurons. Here the ratio of excitatory and inhibitory neurons is set 4:1. ``V_rest`` denotes the resting potential, ``V_th`` denotes the firing threshold, ``V_reset`` denotes the reset value after firing, ``tau`` is the time constant, and ``tau_ref`` is the duration of the refractory period. ``method`` refers to the numerical integration method to be used in simulation. 

Then the synaptic connections between these two groups can be defined:

In [8]:
E2E = bp.models.ExpCOBA(E, E, bp.conn.FixedProb(prob=0.02), E=0., g_max=0.6, tau=5., method='exp_auto')
E2I = bp.models.ExpCOBA(E, I, bp.conn.FixedProb(prob=0.02), E=0., g_max=0.6, tau=5., method='exp_auto')
I2E = bp.models.ExpCOBA(I, E, bp.conn.FixedProb(prob=0.02), E=-80., g_max=0.7, tau=10., method='exp_auto')
I2I = bp.models.ExpCOBA(I, I, bp.conn.FixedProb(prob=0.02), E=-80., g_max=0.7, tau=10., method='exp_auto')

In [10]:
net = bp.dyn.Network(E2E, E2I, I2E, I2I, E=E, I=I)

In [11]:
runner = bp.dyn.DSRunner(net,
                         monitors=['E.spike'],
                         inputs=[('E.input', 20.), ('I.input', 20.)])

In [ ]:
runner(200)
bp.visualize.raster_plot(runner.mon.ts, runner.mon['E.spike'], show=True)